# ⚛️ H₂O — Noiseless VQE Ground State (UCCSD)

This notebook computes the **ground-state energy** of the water molecule (H₂O)
using **noiseless VQE** with a **UCCSD (singles + doubles)** ansatz.

Workflow:
- Build the H₂O Hamiltonian (STO-3G)
- Construct the Hartree–Fock reference state
- Use UCCSD for correlated electron excitations
- Optimize with the **Adam** optimizer
- Plot:
  - Convergence curve
  - Significant basis-state amplitudes

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt

import sys, os, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.vqe_utils import (
    IMG_DIR,
    excitation_ansatz,
    get_optimizer,
    set_seed,
    make_run_config_dict,
    run_signature,
    find_existing_run,
    save_run_record,
    ensure_dirs,
    build_run_filename,
)

ensure_dirs()

# 🧬 Part 1 — Global Configuration

- Basis: STO-3G  
- Electrons: 10  
- Ansatz: UCCSD  
- Optimizer: Adam  
- Device: default.qubit  

In [ ]:
seed = 0
set_seed(seed)

symbols = ["O", "H", "H"]
coordinates = np.array([
    [ 0.000000,  0.000000, 0.000000],
    [ 0.758602,  0.000000, 0.504284],
    [-0.758602,  0.000000, 0.504284],
])

basis = "STO-3G"
electrons = 10

optimizer_name = "Adam"
stepsize = 0.2
max_iterations = 50
ansatz_desc = "UCCSD"

# Build Hamiltonian
hamiltonian, qubits = qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    charge=0,
    basis=basis,
)

print(f"H₂O requires {qubits} qubits.")

hf = qchem.hf_state(electrons=electrons, orbitals=qubits)
print("HF reference state:", "|" + "".join(str(b) for b in hf) + "⟩")

num_wires = qubits

# Excitation indices
singles, doubles = qchem.excitations(electrons=electrons, orbitals=qubits)
print(f"Singles: {len(singles)}, doubles: {len(doubles)}")

# ⚙️ Part 2 — Noiseless VQE QNode (UCCSD)

We build a geometry-specific QNode with:
- HF state  
- UCC singles + doubles  
- Noiseless simulator  
- `parameter-shift` differentiation  

In [ ]:
def get_vqe_qnode(return_state=False):
    """Return a noiseless UCCSD VQE QNode for water."""
    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev, diff_method="parameter-shift")
    def circuit(params):
        excitation_ansatz(
            params,
            wires=range(num_wires),
            hf_state=hf,
            excitations=(singles, doubles),
            excitation_type="both",
        )
        return qml.state() if return_state else qml.expval(hamiltonian)

    return circuit

# 💾 Part 3 — Cache-Aware VQE Driver

Each run is uniquely identified by a **signature hash** combining:
- geometry  
- basis  
- optimizer  
- ansatz  
- iterations  

If a matching run exists, we reload it.

In [ ]:
def run_vqe_with_cache():
    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=optimizer_name,
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=seed,
        noisy=False,
    )

    sig = run_signature(cfg)
    fname = build_run_filename("H2O_Noiseless", optimizer_name, seed, sig)
    existing = find_existing_run(sig)

    # Load cached run
    if existing:
        with open(existing) as f:
            rec = json.load(f)

        print(f"[reuse] Loaded existing run: {existing}")
        energies = rec["energies"]
        params = np.array(rec["final_params"])
        final_state = get_vqe_qnode(return_state=True)(params)
        return energies, params, final_state

    # Fresh run
    n_params = len(singles) + len(doubles)
    cost_fn = get_vqe_qnode(return_state=False)
    opt = get_optimizer(optimizer_name, stepsize=stepsize)

    params = np.zeros(n_params)
    energies = [cost_fn(params)]

    for _ in range(max_iterations):
        params, e_prev = opt.step_and_cost(cost_fn, params)
        energies.append(e_prev)

    energies[-1] = float(cost_fn(params))

    rec = {
        "config": cfg,
        "energies": np.asarray(energies).tolist(),
        "final_params": np.asarray(params).tolist(),
    }
    persisted = save_run_record(fname, rec)

    print(f"[saved] {fname}")
    print(f"[mirrored] {persisted}")

    final_state = get_vqe_qnode(return_state=True)(params)
    return energies, params, final_state

# 🚀 Part 4 — Run the VQE

In [ ]:
energies, params, final_state = run_vqe_with_cache()

# 📉 Part 5 — Convergence Plot

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(energies, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("H₂O VQE Convergence (Noiseless, UCCSD + Adam)")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/H2O_Noiseless_Convergence.png", dpi=300)
plt.show()

print(f"Final ground-state energy: {energies[-1]:.8f} Ha")

# 🔍 Part 6 — Parameter Breakdown (Optional)

In [ ]:
num_singles = len(singles)
theta_s = params[:num_singles]
theta_d = params[num_singles:]

if len(theta_s) > 0:
    print("Example single-excitation parameters:", theta_s[:3])
if len(theta_d) > 0:
    print("Example double-excitation parameters:", theta_d[:3])

# 🧮 Part 7 — Significant Amplitudes in the Final State

In [ ]:
threshold = 1e-2
nz_idx = np.where(np.abs(final_state) > threshold)[0]
nz_amp = final_state[nz_idx]

ket_terms = []
for i, amp in zip(nz_idx, nz_amp):
    bit = f"|{i:0{num_wires}b}⟩"
    if abs(amp.imag) < 1e-6:
        amp_str = f"{amp.real:.4f}"
    else:
        amp_str = f"({amp.real:.4f} + {amp.imag:.4f}j)"
    ket_terms.append(f"{amp_str}{bit}")

ket_str = " + ".join(ket_terms) if ket_terms else "0"
print("Significant components of |ψ⟩:")
print(ket_str)

# 📊 Part 8 — Basis-State Amplitude Plot

In [ ]:
labels = [f"|{i}⟩" for i in nz_idx]
amps = np.abs(nz_amp)

plt.figure(figsize=(12, 5))
plt.bar(labels, amps)
plt.xlabel("Basis state (decimal)")
plt.ylabel("Amplitude")
plt.title("H₂O Ground-State Amplitudes (VQE, Noiseless)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.savefig(f"{IMG_DIR}/H2O_Ground_State.png", dpi=300)
plt.show()